**Проверка версии:**

In [1]:
import sqlalchemy
sqlalchemy.__version__

'2.0.19'

Любое приложение SQLAlchemy начинается с создания движка ([объекта Engine](https://docs.sqlalchemy.org/en/20/tutorial/engine.html#establishing-connectivity-the-engine)).

In [2]:
from sqlalchemy import create_engine
engine = create_engine("postgresql+psycopg2://w9i@localhost/w9i", echo=True)

Функция `create_engine` возвращает нам объект `Engine`.

Объект `Engine` в свою очередь предоставляет нам единицу соединения с БД, называемую [`Connection`](https://docs.sqlalchemy.org/en/20/core/connections.html#sqlalchemy.engine.Connection).

Объект `Connection` *приобретается* путем вызова `Engine.connect()`.

`Connection` предоставляет нам возможность выполнять SQL-запросы, а также управлять транзакциями.

Объект `Connection` представляет собой одно DBAPI-соединение, извлеченное из пула соединений. Для правильного управления пулом соединений необходимо возвращать соединения в пул (т.е. `Connection.close()`) каждый раз, когда соединение не используется.

Поэтому мы будем создавать соединение в контекст менеджере.

In [3]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(
        text("SELECT 'Hello, SQLAlchemy World!'")
    )
    print(result.all())

2023-07-17 03:41:09,433 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-07-17 03:41:09,434 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-07-17 03:41:09,435 INFO sqlalchemy.engine.Engine select current_schema()
2023-07-17 03:41:09,436 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-07-17 03:41:09,438 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-07-17 03:41:09,438 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-07-17 03:41:09,439 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-07-17 03:41:09,440 INFO sqlalchemy.engine.Engine SELECT 'Hello, SQLAlchemy World!'
2023-07-17 03:41:09,440 INFO sqlalchemy.engine.Engine [generated in 0.00082s] {}
[('Hello, SQLAlchemy World!',)]
2023-07-17 03:41:09,441 INFO sqlalchemy.engine.Engine ROLLBACK


Импортированная нами функция `text`, предоставляет нам возможность безопасно писать запросы в *стиле* `SQL`. Эта функция будет более подробно рассмотрена позже.

Результат нашего `SELECT` был возвращен в объекте [Result](https://docs.sqlalchemy.org/en/20/core/connections.html#sqlalchemy.engine.Result), который будет рассмотрен позже, но пока отметим, что этот объект должен использоваться внутри блока "connect" и не передаватся за пределы области действия нашего соединения (контекст менеджера).

Из лога, который выдает нам SQLAlchemy, благодаря флагу `echo=True`, мы можем увидеть что наши запросы **неявно** являются транзакционными, но *незакомиченными* (то есть незафиксированная транзакция).

Коммитить изменения мы должны явно, используя `.commit()` на объекте `Connection`.

Давайте создадим таблицу, внесем какие-то данные и закоммитим изменения.

In [4]:
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))  # Создаем таблицу
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),  # Вставка данных в таблицу
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}],                   # (параметризированный запрос).
    )
    conn.commit()  # Коммит изменений.

2023-07-17 03:41:09,448 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-07-17 03:41:09,448 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2023-07-17 03:41:09,449 INFO sqlalchemy.engine.Engine [generated in 0.00117s] {}
2023-07-17 03:41:09,453 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (%(x)s, %(y)s)
2023-07-17 03:41:09,453 INFO sqlalchemy.engine.Engine [generated in 0.00045s] [{'x': 1, 'y': 1}, {'x': 2, 'y': 4}]
2023-07-17 03:41:09,454 INFO sqlalchemy.engine.Engine COMMIT


Выше мы выполнили два SQL-оператора, которые являются траназкционными.

1. Оператор `CREATE TABLE`
2. Параметризированный оператор `INSERT`

Синтаксис параметризации, будет рассмотрен ниже.

Мы также закоммитили наши изменения, это означает что сейчас в БД у нас таблица `some-table` с внесенными в нее нами данными.

Мы также можем продолжать писать другие запросы, и также вызывать `Connection.commit()` для последующих SQL-операторов.

Существует другой способ фиксации данных, для этого мы можем заранее объявить наш блок connect блоком транзакций.
В этом случае мы используем `Engine.begin()` а не `Engine.connect()`.

In [5]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, {"x": 9, "y": 10}],
    )

2023-07-17 03:41:09,460 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-07-17 03:41:09,461 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (%(x)s, %(y)s)
2023-07-17 03:41:09,462 INFO sqlalchemy.engine.Engine [cached since 0.009108s ago] [{'x': 6, 'y': 8}, {'x': 9, 'y': 10}]
2023-07-17 03:41:09,464 INFO sqlalchemy.engine.Engine COMMIT


Чтобы получить резульирующие строки, мы воспользуемся SQL-оператором `SELECT`.

In [6]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2023-07-17 03:41:09,468 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-07-17 03:41:09,469 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2023-07-17 03:41:09,470 INFO sqlalchemy.engine.Engine [generated in 0.00144s] {}
x: 1  y: 1
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2023-07-17 03:41:09,471 INFO sqlalchemy.engine.Engine ROLLBACK


Выше, были выбраны все строки из нашей таблицы. 

Возвращаемый объект называется `Result` и представляет собой итерируемый объект резульирующих строк.

`Result` имеет множество методов для получения и преобразования резульирующих строк, например, метод `Result.all()`, показанный ранее, который возвращает список всех объектов [Row](https://docs.sqlalchemy.org/en/20/core/connections.html#sqlalchemy.engine.Row). Кроме того, в нем реализован интерфейс итератора Python, что позволяет выполнять итерации непосредственно над коллекцией объектов [Row](https://docs.sqlalchemy.org/en/20/core/connections.html#sqlalchemy.engine.Row).

Сами объекты [Row](https://docs.sqlalchemy.org/en/20/core/connections.html#sqlalchemy.engine.Row) похожи на [именованные кортежи](https://docs.python.org/3/library/collections.html#collections.namedtuple) Python. 

Ниже мы проиллюстрируем различные способы доступа к резульирующим строкам.

```python
# Tuple Assignment :)
result = conn.execute(text("select x, y from some_table"))

for x, y in result:
    ...


# извлечение по индексу
result = conn.execute(text("select x, y from some_table"))

for row in result:
    x = row[0]


# извлечение по атрибуту
result = conn.execute(text("select x, y from some_table"))

for row in result:
    y = row.y

    print(f"Row: {row.x} {y}")


# Mapping Access
result = conn.execute(text("select x, y from some_table"))

for dict_row in result.mappings():
    x = dict_row["x"]
    y = dict_row["y"]
```

Наконец-то параметризированные запросы.

В наших SQL запросах мы каким-то образом должны передавать данные. 
`Connection.execute()` умеет принимать параметры, которые называются [связанные параметры](https://docs.sqlalchemy.org/en/20/glossary.html#term-bound-parameters).

К примеру мы хотим отфильтровать результирующий вывод с помощью ключевого слова `WHERE`.А именно, чтобы в результате вывелись строки, в которых значение колонки `y` больше 2.

Конструкция `text()` принимает наш параметр с ведущим двоеточием `:y`, а фактическое значение параметра передается в качестве словаря, вторым аргументом в `Connection.execute()`.

В примере ниже вместо `:y` подставляется число `2`.

In [7]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table WHERE y > :y"), {"y": 2})
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2023-07-17 03:41:09,498 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-07-17 03:41:09,506 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > %(y)s
2023-07-17 03:41:09,507 INFO sqlalchemy.engine.Engine [generated in 0.00910s] {'y': 2}
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2023-07-17 03:41:09,514 INFO sqlalchemy.engine.Engine ROLLBACK


**Передача нескольких параметров.**

В примерах выше, когда мы разбирали фиксацию (коммит) транзакций, мы выполняли оператор `INSERT`, в котором, как оказалось, можно было одновременно вводить в базу данных несколько строк. Для DML-операторов, таких как `"INSERT"`, `"UPDATE"` и `"DELETE"`, мы можем передать **несколько наборов параметров** в метод `Connection.execute()`, передав список словарей вместо одного словаря, в данном случае, один SQL-оператор будет вызван несколько раз, по одному разу для каждого набора параметров. Такой стиль выполнения известен как [executemany](https://docs.sqlalchemy.org/en/20/glossary.html#term-executemany):

In [8]:
with engine.connect() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 11, "y": 12}, {"x": 13, "y": 14}],
    )
    conn.commit()

2023-07-17 03:41:09,526 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-07-17 03:41:09,532 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (%(x)s, %(y)s)
2023-07-17 03:41:09,536 INFO sqlalchemy.engine.Engine [cached since 0.08349s ago] [{'x': 11, 'y': 12}, {'x': 13, 'y': 14}]
2023-07-17 03:41:09,549 INFO sqlalchemy.engine.Engine COMMIT


Ключевое различие между `"execute"` и `"executemany"` заключается в том, что последний не поддерживает возврат результриующих строк, даже если в операторе присутствует предложение `RETURNING`. Единственным исключением является использование конструкции Core [insert()](https://docs.sqlalchemy.org/en/20/core/dml.html#sqlalchemy.sql.expression.insert), представленной далее в этом учебном пособии в разделе ["Использование утверждений INSERT"](https://docs.sqlalchemy.org/en/20/tutorial/data_insert.html#tutorial-core-insert), которая также указывает на возврат с помощью метода [Insert.returning()](https://docs.sqlalchemy.org/en/20/core/dml.html#sqlalchemy.sql.expression.Insert.returning). В этом случае SQLAlchemy использует специальную логику для реорганизации оператора `INSERT` таким образом, что он может быть вызван для многих строк и при этом поддерживать `RETURNING`.